In [1]:
%matplotlib notebook
#%matplotlib inline
import numpy as np
import sympy
import casadi
import modules as M
import matplotlib.pyplot as plt

In [19]:
def opt_C2(total_time, control_steps, target_path, force_constraints, guess = 0):
    
    control_time0 = total_time / control_steps
    U = casadi.MX.sym("U", control_steps)
    C = casadi.integrator('C','cvodes', ode, 0, control_time0)
    
    # Construct graph of integrator calls
    
    X = x0
    J = 0 #error
    for k in range(control_steps):
        Ck = C(x0=X, p=U[k])
        X = Ck['xf']
        
        J += casadi.sumsqr(M.CT(X) - target_path[k])
        
        # if k == 0:
        #     J += (2 * U[k]) ** 2 / 2
        # else:
        #     J += (U[k] - U[k - 1]) ** 2 
    
    g = [X[4], X[9]]
    target_g = [0, 0]
    
    g.append(X[0] - casadi.arctan(X[7] / X[6]) - np.pi / 2)
    target_g.append(0)
    
    # Allocate an NLP solver
    #nlp = {'x':U, 'f':J}
    nlp = {'x':U, 'f':J, 'g':casadi.vertcat(*g)}
    solver = casadi.nlpsol("solver", "ipopt", nlp)
    
    # Pass bounds, initial guess and solve NLP
    # sol = solver(x0 = CTL_SEQ, lbg = target_g, ubg = target_g)
    umin, umax = force_constraints

    sol = solver(x0 = guess, lbg = target_g, ubg = target_g, lbx = umin, ubx = umax)
    return(sol['x'])


In [20]:
m = 5
x0 = [0, np.pi/2, 0, 0, 0, 0.8, 0, 0, 0, 0.05] #compare to mujoco

control_steps = 20
total_time = 0.3*control_steps

force_constraints = (-50, 50)

target_path  =  np.zeros((control_steps, 3)) 

# get EOM for cyr wheel of mass m and control u.
casadi_EOM, x, u = M.get_casadi_EOM(m)
ode = {}
ode['x'] = x
ode['p'] = u
ode['ode'] = casadi.vertcat(*casadi_EOM)

CTRL_seq = opt_C2(total_time, control_steps, target_path, force_constraints )


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       60
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality c

psetup failed: KeyboardInterrupt
At t = 0.0926102, the setup routine failed in an unrecoverable manner.
Error occured while integrating backward problem # 0
Function fsens20_asens1_C (0x108873e00)
Input 0 (x0): 
[[0.000330012, -0.0375939, -0.00549034, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [1.57217, -0.0981922, -0.0342229, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [0.480007, -0.00084323, -0.000124532, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [0.00219086, -0.156759, -0.0546339, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [0.0068248, -0.195775, -0.225609, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [0.800063, -0.00503557, -0.00146744, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [-0.480007, 0.000758436, 0.000105616, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [-0.00140461, 0.103081, 0.0345531, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 [0.03, 0.135, 0.045, 0, 0, 0, 0, 0, 0, 0